In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
import lightgbm as lgb
import shap
import optuna

# Load dataset
data = pd.read_csv(r"C:\Users\LENOVO FLEX\Downloads\Intrusion Detection System\UNSW_NB15_training-set.csv")

# Drop ID column if present
if 'id' in data.columns:
    data = data.drop(columns=['id'])

# Encode categorical features
categorical_cols = data.select_dtypes(include=['object']).columns
for col in categorical_cols:
    data[col] = LabelEncoder().fit_transform(data[col])

# Separate features and labels
X = data.drop(columns=['attack_cat', 'label'])
y_multi = data['attack_cat']  # Multiclass labels
y_binary = data['label']  # Binary labels

# Normalize numerical features
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

# Feature Selection using LightGBM + SHAP
lgb_model = lgb.LGBMClassifier()
lgb_model.fit(X_scaled, y_binary)
explainer = shap.Explainer(lgb_model)
shap_values = explainer(X_scaled)

# Get top 15 important features
important_features = np.argsort(np.abs(shap_values.values).mean(axis=0))[-15:]  # Top 15 features
X_selected = X_scaled[:, important_features]

# Print the selected features
feature_names = X.columns[important_features]
print("Selected Features after LightGBM + SHAP:")
print(feature_names)

# Train-test split for binary and multiclass
X_train_bin, X_test_bin, y_train_bin, y_test_bin = train_test_split(X_selected, y_binary, test_size=0.2, stratify=y_binary)
X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(X_selected, y_multi, test_size=0.2, stratify=y_multi)

# Handle class imbalance using SMOTE
smote = SMOTE()
X_train_bin, y_train_bin = smote.fit_resample(X_train_bin, y_train_bin)
X_train_multi, y_train_multi = smote.fit_resample(X_train_multi, y_train_multi)

# Reshape for LSTM input (for multiclass model)
X_train_multi_lstm = np.expand_dims(X_train_multi, axis=1)
X_test_multi_lstm = np.expand_dims(X_test_multi, axis=1)

# Compute class weights for multiclass model
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_multi), y=y_train_multi)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

# Multiclass Model (LSTM with Attention)
def create_multiclass_model(trial):
    model = Sequential()
    model.add(Input(shape=(X_train_multi_lstm.shape[1], X_train_multi_lstm.shape[2])))

    # First Bidirectional LSTM layer
    model.add(Bidirectional(LSTM(units=trial.suggest_int("units_1", 64, 256, step=32), return_sequences=True)))
    model.add(BatchNormalization())
    model.add(Dropout(trial.suggest_float("dropout_1", 0.2, 0.5)))

    # Second Bidirectional LSTM layer
    model.add(Bidirectional(LSTM(units=trial.suggest_int("units_2", 32, 128, step=16))))
    model.add(BatchNormalization())
    model.add(Dropout(trial.suggest_float("dropout_2", 0.2, 0.5)))

    # Output layer
    model.add(Dense(len(np.unique(y_train_multi)), activation='softmax'))

    # Compile model
    optimizer = Adam(learning_rate=trial.suggest_loguniform("lr", 1e-4, 1e-2))
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

# Hyperparameter Optimization for Multiclass Model
def objective(trial):
    model = create_multiclass_model(trial)
    history = model.fit(X_train_multi_lstm, y_train_multi, validation_data=(X_test_multi_lstm, y_test_multi),
                        epochs=12, batch_size=trial.suggest_categorical("batch_size", [32, 64, 128]),
                        class_weight=class_weights_dict, verbose=0,
                        callbacks=[EarlyStopping(monitor='val_loss', patience=3)])
    return max(history.history['val_accuracy'])

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Train Best Multiclass Model
best_params = study.best_params
multiclass_model = create_multiclass_model(optuna.trial.FixedTrial(best_params))
multiclass_model.fit(X_train_multi_lstm, y_train_multi, validation_data=(X_test_multi_lstm, y_test_multi),
                     epochs=50, batch_size=best_params['batch_size'], class_weight=class_weights_dict,
                     callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

# Evaluate Multiclass Model
multiclass_accuracy = multiclass_model.evaluate(X_test_multi_lstm, y_test_multi)[1] * 100
print(f'Multiclass Model Accuracy: {multiclass_accuracy:.2f}%')

# Save Multiclass Model
multiclass_model.save('/content/sample_data/multiclass_model.h5')
print("Multiclass Model saved successfully.")




# Binary Classification Model (DNN)
binary_model = Sequential([
    Dense(128, activation='selu', input_shape=(X_train_bin.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='selu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])
binary_model.compile(optimizer=AdamW(), loss=BinaryCrossentropy(), metrics=['accuracy'])

# Train Binary Model
binary_model.fit(X_train_bin, y_train_bin, validation_data=(X_test_bin, y_test_bin), epochs=20, batch_size=32)

# Evaluate Binary Model
binary_loss, binary_acc = binary_model.evaluate(X_test_bin, y_test_bin)
print(f'Binary Model Accuracy: {binary_acc * 100:.2f}%')

# Save Binary Model
binary_model.save('/content/sample_data/binary_model.h5')
print("Binary Model saved successfully.")